In [28]:
import pandas as pd
import numpy as np
from itertools import combinations
from pprint import pprint

In [29]:
transaction = pd.read_excel('bread.xlsx',sheet_name='Sheet2')


In [35]:
t = {i : set() for i in transaction['Transaction'].unique()}
for i in range(len(transaction)) :
    row = transaction.iloc[i]
    t[row['Transaction']].add(row['Item'])
tid = transaction['Transaction'].unique()

{1: {'Bread'},
 2: {'Scandinavian'},
 3: {'Jam', 'Hot chocolate', 'Cookies'},
 4: {'Muffin'},
 5: {'Pastry', 'Coffee', 'Bread'},
 6: {'Pastry', 'Muffin', 'Medialuna'},
 7: {'Pastry', 'Medialuna', 'Tea', 'Coffee'},
 8: {'Pastry', 'Bread'},
 9: {'Muffin', 'Bread'},
 10: {'Scandinavian', 'Medialuna'},
 11: {'NONE', 'Medialuna', 'Bread'},
 12: {'Jam', 'Coffee', 'Pastry', 'Tea', 'Tartine'},
 13: {'Coffee', 'Bread', 'Basket'},
 14: {'Pastry', 'Medialuna', 'Bread'},
 15: {'NONE', 'Mineral water', 'Scandinavian'},
 16: {'Medialuna', 'Coffee', 'Bread'},
 17: {'Hot chocolate'},
 18: {'Farm House'},
 19: {'Farm House', 'Bread'},
 20: {'Medialuna', 'Bread'},
 21: {'Medialuna', 'Coffee', 'Bread'},
 22: {'Jam'},
 23: {'Muffin', 'Scandinavian'},
 24: {'Bread'},
 25: {'Scandinavian'},
 26: {'Fudge'},
 27: {'Scandinavian'},
 28: {'Coffee', 'Bread'},
 29: {'NONE', 'Jam', 'Bread'},
 30: {'Bread'},
 31: {'Basket'},
 32: {'Muffin', 'Scandinavian'},
 33: {'Coffee'},
 34: {'Muffin', 'Coffee'},
 35: {'Muffin'

In [31]:
def generate(data , msp) :
    sup , support = dict() , dict()
    items = data['Item'].unique()
    idx = []
    for j in range(len(items)) :
        if items[j] == 'None' or items[j] == 'NONE' :
            idx.append(j)
    for i in idx : 
        items = np.delete(items,i)
    for iteration in range(1,4):
        subset = combinations(items,iteration)
        items = set()
        for j in list(subset) :
            cnt , j = 0 , set(j)
            for i in t : 
                if j.issubset(t[i]) : cnt += 1
            if cnt == 0 : break
            supcnt = cnt/ len(t)
            if supcnt > msp :
                items = set(items) | set(j)
                sup[tuple(j)] = cnt
                support[tuple(j)] = supcnt
    return pd.DataFrame(list(sup.items()) , columns=["Items" , "Support"]) , support

In [32]:
freq_item_dataset, support = generate(transaction , 0.04)
freq_item_dataset

,Items,Support
0,"(Bread,)",3097
1,"(Hot chocolate,)",552
2,"(Cookies,)",515
3,"(Coffee,)",4528
4,"(Pastry,)",815
5,"(Medialuna,)",585
6,"(Tea,)",1350
7,"(Cake,)",983
8,"(Sandwich,)",680
9,"(Pastry, Coffee)",450


In [33]:
def find_relation(df , confidence) :
    ans = []
    relations = dict()
    for i in range(len(df)) :
        current = df.iloc[i]
        if len(current['Items']) > 1 : 
            relations[current['Items']] = current['Support']
    for i in relations : 
        f , d1  , d2 =  0 ,0 , 0
        for k in t : 
            if set(i).issubset(t[k]) : 
                f+=1
            if i[0] in t[k] : d1 += 1
            if i[1] in t[k] : d2 += 1
        if f/d1 > confidence : 
            ans.append([i[0],i[1],f/d1])
        if f/d2 > confidence : 
            ans.append([i[1],i[0],f/d2])
    return ans

In [34]:
association_rules = find_relation(freq_item_dataset , 0.5)
for i,j,k in association_rules : 
    print(i + " -> " + j)
    print("Confidence : " , str(k))

Pastry -> Coffee
Confidence :  0.5521472392638037
Cake -> Coffee
Confidence :  0.5269582909460834
